# 01 – Stain Normalization

This notebook demonstrates how to estimate stain vectors and normalize a histology image using the `StainNormalizer` class.  Edit the `image_path` variable below to point to your image file.

In [ ]:
import cv2
import matplotlib.pyplot as plt
from isletscope.stain import StainNormalizer

# Path to your image file
image_path = 'path/to/your/image.tif'

# Load image (OpenCV loads in BGR format)
img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f'Cannot load image at {image_path}')

# Instantiate normalizer and estimate stain matrix
normalizer = StainNormalizer()
stain_matrix = normalizer.estimate_stain_matrix(img)
print('Estimated stain matrix:')
print(stain_matrix)

# Normalize the image using the estimated matrix
img_norm = normalizer.normalize(img)

# Display original and normalized images
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[0].axis('off')
axes[1].imshow(cv2.cvtColor(img_norm, cv2.COLOR_BGR2RGB))
axes[1].set_title('Normalized')
axes[1].axis('off')
plt.show()